<a href="https://colab.research.google.com/github/moxie77/Amazon_sentiment_analysis/blob/main/Amazon_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

In [3]:
dataset = pd.read_csv('/content/drive/MyDrive/Reviews.csv')

In [4]:
dataset.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
dataset['Score'].unique()

array([5, 1, 4, 2, 3])

In [6]:
filtered_data = dataset[dataset['Score']!=3]

In [7]:
filtered_data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [8]:
filtered_data['Score'].unique()

array([5, 1, 4, 2])

In [9]:
def share_group(score):
  if score < 3:
    return 0
  else:
    return 1

In [10]:
actual_score = filtered_data['Score']
positive_negative = actual_score.map(share_group)
filtered_data['Score'] = positive_negative

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
filtered_data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...


In [12]:
sorted_data = filtered_data.sort_values('ProductId',axis=0,ascending=True,inplace=False,kind='quicksort',na_position='last')

In [13]:
sorted_data.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
150523,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,1,939340800,EVERY book is educational,this witty little book makes my son laugh at l...
150505,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,1,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
150506,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,1,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...
150507,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,1,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...
150508,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,1,1018396800,A great way to learn the months,This is a book of poetry about the months of t...


In [14]:
second_sort = sorted_data.drop_duplicates(subset={'UserId','ProfileName','Time','Text'},keep='first',inplace=False)

In [15]:
full_sort = second_sort[second_sort['HelpfulnessNumerator']<=second_sort['HelpfulnessDenominator']]

In [16]:
full_sort['Score'].value_counts()

1    307061
0     57110
Name: Score, dtype: int64

In [17]:
import re
from bs4 import BeautifulSoup


def decontracted(phrase):

  # specific
  phrase = re.sub(r"won't", "will not", phrase)
  phrase = re.sub(r"can\'t", "can not", phrase)

  # general
  phrase = re.sub(r"n\'t", " not", phrase)
  phrase = re.sub(r"\'re", " are", phrase)
  phrase = re.sub(r"\'s", " is", phrase)
  phrase = re.sub(r"\'d", " would", phrase)
  phrase = re.sub(r"\'ll", " will", phrase)
  phrase = re.sub(r"\'t", " not", phrase)
  phrase = re.sub(r"\'ve", " have", phrase)
  phrase = re.sub(r"\'m", " am", phrase)
  
  return phrase

In [18]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",
                "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 
                'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',
                'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 
                'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 
                'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 
                'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',
                'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',
                'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
                'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very',
                's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're',
                've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',
                "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',
                "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't",
                'won', "won't", 'wouldn', "wouldn't"])

In [19]:
from tqdm import tqdm

processed_reviews = []

for sentence in tqdm(full_sort['Text'].values):
  sentence = re.sub('http\S+','',sentence)
  sentence = BeautifulSoup(sentence, 'lxml').get_text()
  sentence = decontracted(sentence)
  sentence = re.sub("\S*\d\S*","",sentence).strip()
  sentence = re.sub('[^A-Za-z]+','',sentence)
  sentence = ' '.join(e.lower() for e in sentence.split() if e.lower() not in stopwords)
  processed_reviews.append(sentence.strip())

100%|██████████| 364171/364171 [01:34<00:00, 3843.15it/s]


In [20]:
processed_reviews

['...',
 'samepriceasdrfostersmith',
 '...',
 'beetlejuice',
 '...',
 'ifirststartedusingthe',
 'iwasgettingbittenupeverynightiboughtoftheseunitsandputoneonthefloorbymybedandoneundermylivingroomcouchiwasamazedtoseehowmanyfleesendeduponthepadifoundthebestresultswhenikeptmydooropenlettingcoldairinmyhouseitwasaboutdegreesandthetrapsbecameextremelycongestedwithfleasinthatoneday',
 'iwashopefulthatthisfleatrapwouldbejustthethingnotoxicspraysbugbalmsetcbutpluggeditinandnothingnotonefleawascaughtasamatteroffactnothingwascaughtsoicanassumeeithertwothingsitdoesnotworkoridonothaveanyfleasandallthestuffiusedtogetridofthembeforhandkilledthemoff',
 'ihavebeenusingthetrapsforfewweeksnowandtriedalltheroomsinthehousesofaritdidnotevengetfleaonlycollectingdust',
 'anyonewhohasapetcanalsogetfleasinthehomethevictorfleatrapisfantasticihaveoneineachroomandtheyeradicateanyfleasthatmypetormyselfbringsinfromoutside',
 'mydogsgotflealatelyisawthisproductandgaveitatryitworksitcaughtfleasandothersmallinsetsatthef

In [21]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
token = Tokenizer()

In [22]:
token.fit_on_texts(processed_reviews)
v_size = len(token.word_index) + 1
encoded_review = token.texts_to_sequences(processed_reviews)
max_length = 150
review_pad = pad_sequences(encoded_review,maxlen=max_length,padding='post')

In [23]:
print(len(review_pad))
print(review_pad.shape)

364171
(364171, 150)


In [24]:
(0.9*len(review_pad))

327753.9

In [25]:
X_train = review_pad[0:int(0.9*len(review_pad))]
X_test = review_pad[int(0.9*len(review_pad)):len(review_pad)]

In [26]:
print(X_train.shape)
print(X_test.shape)

(327753, 150)
(36418, 150)


In [27]:
y_train = np.array(list(full_sort['Score'].iloc[0:int(0.9*len(review_pad))].values))
y_test = np.array(list(full_sort['Score'].iloc[int(0.9*len(review_pad)):len(review_pad)].values))

In [28]:
print(y_train.shape, y_test.shape)

(327753,) (36418,)


In [29]:
from keras.models import Sequential
from keras.layers import Input,Dense, LSTM
from keras.layers.embeddings import Embedding
import keras.backend as K

In [30]:
model = Sequential()

In [31]:
model.add(Embedding(v_size,input_length= max_length,output_dim=1))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss=['binary_crossentropy'],optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 1)            362275    
_________________________________________________________________
lstm (LSTM)                  (None, 150, 128)          66560     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 564,580
Trainable params: 564,580
Non-trainable params: 0
_________________________________________________________________


In [32]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=5, patience=5)

In [33]:
history = model.fit(X_train, y_train, epochs=50, batch_size=1024, validation_split=0.1, callbacks=[es] )

Epoch 1/50
289/289 [==============================] - 32s 83ms/step - loss: 0.4629 - accuracy: 0.8316 - val_loss: 0.4574 - val_accuracy: 0.8302
Epoch 2/50
289/289 [==============================] - 23s 80ms/step - loss: 0.4307 - accuracy: 0.8455 - val_loss: 0.4560 - val_accuracy: 0.8302
Epoch 3/50
289/289 [==============================] - 23s 79ms/step - loss: 0.4299 - accuracy: 0.8461 - val_loss: 0.4575 - val_accuracy: 0.8302
Epoch 4/50
289/289 [==============================] - 23s 80ms/step - loss: 0.4305 - accuracy: 0.8456 - val_loss: 0.4565 - val_accuracy: 0.8302
Epoch 5/50
289/289 [==============================] - 23s 79ms/step - loss: 0.4300 - accuracy: 0.8459 - val_loss: 0.4564 - val_accuracy: 0.8302
Epoch 6/50
289/289 [==============================] - 23s 80ms/step - loss: 0.4307 - accuracy: 0.8455 - val_loss: 0.4564 - val_accuracy: 0.8302
Epoch 7/50
289/289 [==============================] - 23s 79ms/step - loss: 0.4288 - accuracy: 0.8465 - val_loss: 0.4566 - val_accuracy:

In [36]:
prob = model.predict(X_test)
y_pred = []
for i in range(prob.shape[0]):
  if(prob[i][0]>=0.5):
    y_pred.append(1)
  else:
    y_pred.append(0)


In [38]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      6111
           1       0.83      1.00      0.91     30307

    accuracy                           0.83     36418
   macro avg       0.42      0.50      0.45     36418
weighted avg       0.69      0.83      0.76     36418



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
from keras.models import Sequential
from keras.layers import Input,LSTM, Dense, Bidirectional
from keras.layers.embeddings import Embedding

In [40]:
model_2 = Sequential()

In [41]:
model_2.add(Embedding(v_size,input_length=max_length,output_dim=1))
model_2.add(Bidirectional(LSTM(128, return_sequences=True)))
model_2.add(Bidirectional(LSTM(128)))
model_2.add(Dense(32,activation='relu'))
model_2.add(Dense(1,activation='sigmoid'))
model_2.compile(loss=['binary_crossentropy'],optimizer='adam',metrics=['accuracy'])
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 1)            362275    
_________________________________________________________________
bidirectional (Bidirectional (None, 150, 256)          133120    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               394240    
_________________________________________________________________
dense_2 (Dense)              (None, 32)                8224      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 897,892
Trainable params: 897,892
Non-trainable params: 0
_________________________________________________________________


In [45]:
history = model_2.fit(X_train, y_train, epochs=50, batch_size=1024, validation_split=0.1, callbacks=[es] )

Epoch 1/50
289/289 [==============================] - 59s 189ms/step - loss: 0.4532 - accuracy: 0.8477 - val_loss: 0.4558 - val_accuracy: 0.8302
Epoch 2/50
289/289 [==============================] - 54s 185ms/step - loss: 0.4281 - accuracy: 0.8459 - val_loss: 0.4996 - val_accuracy: 0.8302
Epoch 3/50
289/289 [==============================] - 54s 185ms/step - loss: 0.2660 - accuracy: 0.9043 - val_loss: 0.9552 - val_accuracy: 0.6847
Epoch 4/50
289/289 [==============================] - 53s 185ms/step - loss: 0.0383 - accuracy: 0.9865 - val_loss: 2.3249 - val_accuracy: 0.5464
Epoch 5/50
289/289 [==============================] - 53s 185ms/step - loss: 0.0043 - accuracy: 0.9994 - val_loss: 3.1814 - val_accuracy: 0.5410
Epoch 6/50
289/289 [==============================] - 54s 186ms/step - loss: 0.0036 - accuracy: 0.9995 - val_loss: 3.3635 - val_accuracy: 0.5395
Epoch 00006: early stopping


In [46]:
prob_yy = model_2.predict(X_test)

In [47]:
y_pred2 = []
for i in range(prob_yy.shape[0]):
  if(prob_yy[i][0]>=0.5):
    y_pred2.append(1)
  else:
    y_pred2.append(0)

In [48]:
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.16      0.43      0.24      6111
           1       0.83      0.56      0.67     30307

    accuracy                           0.54     36418
   macro avg       0.50      0.49      0.45     36418
weighted avg       0.72      0.54      0.59     36418

